# Przewidywanie zwycięzcy rundy w grze Counter Strike: Global Offensive

# Dane

Zbiór danych składa się ze snapshotów rund z około 700 meczów z profesjonalnych turniejów
rozgrywanych w 2019 i 2020 roku. Snapshoty, czyli zestawienie pewnych stanów kluczowych ele-
mentów rozgrywki, były rejestrowane podczas gry co 20 sekund aż do rozstrzygnięcia danej rundy.
Łączna liczba zapisanych snapshotów wynosi 122411. Część tych rekordów będzie traktowana jako
zbiór danych uczących, a pozostała część jako część danych testów. Każdy rekord traktowany jest
jako pojedynczy, niezależny element do analizy danych.

# Czym jest klasyfikator MLP



# Import bibliotek i danych

# Przeprocesowanie danych